In [154]:
def readFeaturesWithSums(jsonTest,test):
    res=[]
    res.append(['match_id','r_xp_sum','d_xp_sum','r_gold_sum','d_gold_sum','r_lh_sum','d_lh_sum'])
    with bz2.BZ2File(jsonTest) as matches_file:
        for line in matches_file:
            match = json.loads(line)
            r_xp_sum=0
            d_xp_sum=0
            r_gold_sum=0
            d_gold_sum=0
            r_lh_sum=0
            d_lh_sum=0
            for i in range(len(match['times'])):
                if(match['times'][i]<5*60):
                    r_xp_sum+=match['players'][0]['xp_t'][i]
                    r_xp_sum+=match['players'][1]['xp_t'][i]
                    r_xp_sum+=match['players'][2]['xp_t'][i]
                    r_xp_sum+=match['players'][3]['xp_t'][i]
                    r_xp_sum+=match['players'][4]['xp_t'][i]
        
                    d_xp_sum+=match['players'][5]['xp_t'][i]
                    d_xp_sum+=match['players'][6]['xp_t'][i]
                    d_xp_sum+=match['players'][7]['xp_t'][i]
                    d_xp_sum+=match['players'][8]['xp_t'][i]
                    d_xp_sum+=match['players'][9]['xp_t'][i]
        
                    r_gold_sum+=match['players'][0]['gold_t'][i]
                    r_gold_sum+=match['players'][1]['gold_t'][i]
                    r_gold_sum+=match['players'][2]['gold_t'][i]
                    r_gold_sum+=match['players'][3]['gold_t'][i]
                    r_gold_sum+=match['players'][4]['gold_t'][i]
        
                    d_gold_sum+=match['players'][5]['gold_t'][i]
                    d_gold_sum+=match['players'][6]['gold_t'][i]
                    d_gold_sum+=match['players'][7]['gold_t'][i]
                    d_gold_sum+=match['players'][8]['gold_t'][i]
                    d_gold_sum+=match['players'][9]['gold_t'][i]
     
                    r_lh_sum+=match['players'][0]['lh_t'][i]
                    r_lh_sum+=match['players'][1]['lh_t'][i]
                    r_lh_sum+=match['players'][2]['lh_t'][i]
                    r_lh_sum+=match['players'][3]['lh_t'][i]
                    r_lh_sum+=match['players'][4]['lh_t'][i]
        
                    d_lh_sum+=match['players'][5]['lh_t'][i]
                    d_lh_sum+=match['players'][6]['lh_t'][i]
                    d_lh_sum+=match['players'][7]['lh_t'][i]
                    d_lh_sum+=match['players'][8]['lh_t'][i]
                    d_lh_sum+=match['players'][9]['lh_t'][i]
            res.append([match['match_id'],r_xp_sum,d_xp_sum,r_gold_sum,d_gold_sum,r_lh_sum,d_lh_sum])
    import csv
    with open('tmp.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerows(res)
    features = pandas.read_csv(test)
    s = pandas.read_csv('./sums.csv', index_col='match_id')
    result = pandas.merge(features, s, left_on='match_id', right_index=True, how='left', sort=False)
    return result

In [170]:
#test=readFeaturesWithSums('matches_test.jsonlines.bz2','features_test.csv')
#features=readFeaturesWithSums('matches.jsonlines.bz2','features.csv')
features=pandas.read_csv('features.csv')
test=pandas.read_csv('features_test.csv')
print('yep!')
features.fillna(9999999,inplace=True)
test.fillna(9999999,inplace=True)
y=features['radiant_win']
features=features.drop(['duration','radiant_win','tower_status_radiant',
               'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
clf=GradientBoostingClassifier(n_estimators=30, random_state=42)
clf.fit(features,y)

yep!


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=30, presort='auto', random_state=42,
              subsample=1.0, verbose=0, warm_start=False)

array([ 0.46936631,  0.53440983,  0.50510758,  0.52275224,  0.4760958 ,
        0.510723  ,  0.50925907,  0.52607492,  0.47088511,  0.49669866,
        0.49011569,  0.51625118,  0.4588971 ,  0.49114043,  0.57201441,
        0.49804318,  0.46314419,  0.51295109,  0.51997051,  0.48339997,
        0.51259305,  0.54845578,  0.48865242,  0.51040166,  0.5327513 ,
        0.50090307,  0.49566761,  0.46267691,  0.45390498,  0.56978954,
        0.51993691,  0.57606385,  0.47210817,  0.52027642,  0.5146185 ,
        0.55513407,  0.46730444,  0.52196057,  0.50481735,  0.46081351,
        0.49800841,  0.52215735,  0.49837351,  0.52912486,  0.50675005,
        0.54737964,  0.48442401,  0.4672157 ])

In [171]:
pred=clf.predict_proba(test)[:,1]
match_id=pandas.read_csv('./features_test.csv')['match_id']
total=[]
total.append(["match_id","radiant_win"])
for i in range(0,len(match_id)):
    total.append([match_id[i],pred[i]])
import csv
with open('res.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(total)

In [153]:
test

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,r_xp_sum,d_xp_sum,r_gold_sum,d_gold_sum,r_lh_sum,d_lh_sum
0,1430287923,0,93,4,1103,1089,8,0,1,9,...,4,2,0,118.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
1,1430293357,1,20,2,556,570,1,0,0,9,...,2,2,1,16.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
2,1430301774,1,112,2,751,808,1,0,0,13,...,3,3,0,-34.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
3,1430323933,1,27,3,708,903,1,1,1,11,...,4,2,1,-26.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
4,1430331112,1,39,4,1259,661,4,0,0,9,...,3,2,1,-33.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
5,1430334264,0,97,5,2067,1549,15,0,0,8,...,5,2,0,34.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
6,1430334995,1,97,4,1862,1487,24,0,1,5,...,3,2,1,-32.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
7,1430349466,1,106,5,1991,1983,19,2,0,10,...,5,2,1,-35.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
8,1430366484,1,57,2,517,500,0,0,1,9,...,3,2,2,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0
9,1430375099,7,36,3,1060,691,5,0,0,6,...,2,2,0,-27.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0,9999999.0


In [122]:
test.columns.values

array(['match_id', 'start_time', 'lobby_type', 'r1_hero', 'r1_level',
       'r1_xp', 'r1_gold', 'r1_lh', 'r1_kills', 'r1_deaths', 'r1_items',
       'r2_hero', 'r2_level', 'r2_xp', 'r2_gold', 'r2_lh', 'r2_kills',
       'r2_deaths', 'r2_items', 'r3_hero', 'r3_level', 'r3_xp', 'r3_gold',
       'r3_lh', 'r3_kills', 'r3_deaths', 'r3_items', 'r4_hero', 'r4_level',
       'r4_xp', 'r4_gold', 'r4_lh', 'r4_kills', 'r4_deaths', 'r4_items',
       'r5_hero', 'r5_level', 'r5_xp', 'r5_gold', 'r5_lh', 'r5_kills',
       'r5_deaths', 'r5_items', 'd1_hero', 'd1_level', 'd1_xp', 'd1_gold',
       'd1_lh', 'd1_kills', 'd1_deaths', 'd1_items', 'd2_hero', 'd2_level',
       'd2_xp', 'd2_gold', 'd2_lh', 'd2_kills', 'd2_deaths', 'd2_items',
       'd3_hero', 'd3_level', 'd3_xp', 'd3_gold', 'd3_lh', 'd3_kills',
       'd3_deaths', 'd3_items', 'd4_hero', 'd4_level', 'd4_xp', 'd4_gold',
       'd4_lh', 'd4_kills', 'd4_deaths', 'd4_items', 'd5_hero', 'd5_level',
       'd5_xp', 'd5_gold', 'd5_lh', 'd5_kills